In [1]:
using JuMP
using Gurobi

## Sets

In [2]:
I=2 # 1=gyn og 2=sarkom
U=1 # 1=hast (og 2=ikke-hast)
S=1 # 1 scanner
D=H=5 # 5 dager
R=2 # 2 radiologer
M=4 # 1=tolking, 2=åpen, 3=scanner, 4=MDT

;

## Parameters

In [3]:
#i rader, m kolonner.
T_im=[[1 1 1 1];
      [1 1 1 1]]

G_d=[3 3 3 3 3]

N_sd=[8 8 8 8 8]

S_sd=[0 0 0 0 1]

D_iuW=[5 5]

K_dr=[[8 8];
      [8 8];
      [8 8];
      [8 8];
      [8 8];]

D_iuD=[1 1]

B_iu=[5 5]

L_d=[1 1 1 1 1]

A_id=[[1 0 0 0 0];
      [0 0 1 0 0]]

C=3 # maks antall scannedager en radiolog kan være satt til
;

## Variables

In [4]:
m = Model(with_optimizer(Gurobi.Optimizer))
@variable(m, delta_p[1:D] >= 0, Int)
@variable(m, delta_m[1:D] >= 0, Int)
@variable(m, x[1:I,1:U,1:S,1:D] >= 0, Int)
@variable(m, w[1:I,1:U,1:D,1:R,1:M] >= 0, Int)
@variable(m, u_i[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, v[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, alpha[1:I,1:U,1:D], Bin)
@variable(m, beta[1:D,1:R], Bin)
@variable(m, gamma[1:I,1:D,1:R], Bin)
@variable(m, zeta[1:D]>=0, Int)#number of open slots at the scanner at day d
;

Academic license - for non-commercial use only


## Objective function

In [5]:
@objective(m, Min, sum(delta_p+delta_m));

## Constraints

In [6]:
#Constraint2
@constraint(m,[d=1:D], delta_m[d]-delta_p[d] + sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, r=1:R, m=2)==G_d[d])             
                 
#Constraint3
@constraint(m,[s=1:S,d=1:D], sum(x[i,u,s,d] for i=1:I,u=1:U)+zeta[d]==N_sd[s,d]-S_sd[s,d])
                            
#Constraint4
@constraint(m,[i=1:I,u=1:U], sum(x[i,u,s,d] for s=1:S,d=1:D)==D_iuW[i])

#Constraint5
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)==K_dr[d,r])

#Constraint6
@constraint(m,[i=1:I,u=1:U], sum(alpha[i,u,d] for d=1:D)>=D_iuD[i])

#Constraint7
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)>=alpha[i,u,d])

#Constraint8
@constraint(m,[i=1:I,h=1:H], sum(u_i[i,u,d,h,r] for m=1, u=1:U, r=1:R, d=1:D if d<=h)==sum(w[i,u,h,r,m] for u=1:U, r=1:R, m=1))                                

#Constraint10
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)-sum(u_i[i,u,d,h,r] for r=1:R, h=(d:d+B_iu[i]) if h<=D)==0)

#Constraint12
@constraint(m,[d=1:D], sum(w[i,u,d,r,m] for i=1:I, u=1:U, r=1:R, m=3)==sum(N_sd[s,d] for s=1:S))

#Constraint13
@constraint(m,[d=1:D], sum(beta[d,r] for r=1:R)==L_d[d])
                
#Constraint14
@constraint(m,[d=1:D, r=1:R], K_dr[d,r]*(1-beta[d,r])-sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I, u=1:U, m =1:M if m!=3)==0)

#Constraint15
@constraint(m, [i=1:I,d=1:D], sum(w[i,u,d,r,m] for u=1:U, r=1:R, m=4)==T_im[i,4]*A_id[i,d])
                
#Constraint16
@constraint(m,[i=1:I,d=1:D],sum(gamma[i,d,r] for r=1:R)==A_id[i,d])
                
#Constraint17
@constraint(m,[i=1:I,d=1:D, r=1:R], sum(w[i,u,d,r,m] for u=1:U, m=4)==gamma[i,d,r])               

#Egendefinert. Radiologer kan max være med på 3 scannedager i uka
@constraint(m,[r=1:R], sum(beta[d,r] for d=1:D)<=C)  
                                
;

                                                

## Results

In [7]:

@time begin
    optimize!(m)
    println("Run time: ")
end

println("Status: ",termination_status(m))
println("Primal status: ",primal_status(m))
println("Dual status: ",dual_status(m))
#println("Dual con1: ", shadow_price(con1[1]))
println("Objective value: ", objective_value(m))

println()
println("****** Variable: x ******")
for u=1:U
    for s=1:S
        for d=1:D
            println()
            println("Day: ",d)
            println("__________________")
            for i=1:I
            if value(x[i,u,s,d]) != 0
                println("x[",i,u,s,d,"] = ", value(x[i,u,s,d]))
            end
                if i==2
                    println("Open slots = ",value(zeta[d]))
                end
            end
        end
    end
end
println()
println("****** Variable: w ******")
for d=1:D
    println()
    println("Day: ",d)
    println("__________________")
    for i=1:I
        #println("Patient group: ", i)
        for u=1:U
            for r=1:R
                #println("Radioligst: ",r)
                for m=1:M
                    if value(w[i,u,d,r,m]) != 0
                        println("w[",i,u,d,r,m,"] = ", value(w[i,u,d,r,m]))
                    end
                end
            end
        end
    end
end
println()
println("****** Variable: u ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(u_i[i,u,d,h,r]) != 0
                        println("u[",i,u,d,h,r,"] = ", value(u_i[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: v ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(v[i,u,d,h,r]) != 0
                        println("v[",i,u,d,h,r,"] = ", value(v[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: delta ******")
for d =1:D
    println()
    println("Day: ",d)
    println("__________________")
    if value(delta_p[d]) !=0
        println("delta_p[",d,"] = ",value(delta_p[d]))
    end
    if value(delta_m[d])!=0
        println("delta_m[",d,"] = ",value(delta_m[d]))
    end
end

println()
println("****** Variable: alpha ******")
for i=1:I
    for u=1:U
        for d=1:D
            if value(alpha[i,u,d]) != 0
                println("alpha[",i,u,d,"] = ", value(alpha[i,u,d]))
            end
        end
    end
end

println()
println("****** Variable: beta ******")
for d=1:D
    for r=1:R
        if value(beta[d,r]) != 0
            println("beta[",d,r,"] = ", value(beta[d,r]))
        end
    end
end

println()
println("****** Variable: gamma ******")
for i=1:I
    for d=1:D
        for r=1:R
            if value(gamma[i,d,r]) != 0
                println("gamma[",i,d,r,"] = ", value(gamma[i,d,r]))
            end
        end
    end
end

Academic license - for non-commercial use only
Optimize a model with 106 rows, 345 columns and 425 nonzeros
Variable types: 0 continuous, 345 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 30.0000000
Presolve removed 87 rows and 325 columns
Presolve time: 0.00s
Presolved: 19 rows, 20 columns, 50 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 20 integer (10 binary)

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Run time: 
 11.402450 seconds (43.55 M allocations: 2.197 GiB, 10.31% gc time)
Status: OPTIMAL
Primal status: FEASIBLE_POINT
Dual status: NO_SOLUTION
Objective value: 0.0

*****

## The model

In [8]:
#print(m)